In [78]:
import pandas as pd
import numpy as np
#from CRPS import CRPS
from sklearn.metrics import mean_pinball_loss

In [79]:
predictions_model1 = pd.read_csv("../../results/predictions_model1.csv")
predictions_model2 = pd.read_csv("../../results/predictions_model2.csv")
predictions_model3 = pd.read_csv("../../results/predictions_model3.csv")
predictions_GB = pd.read_csv("../../results/predictions_GB.csv")
predictions_NN = pd.read_csv("../../results/predictions_NN.csv")

In [80]:
# import data
hour = 17
data = pd.read_csv(f"../../data/final_{hour}.csv")
data["datetime"] = pd.to_datetime(data["datetime"])

In [81]:
# cleanup
data.drop(["loadFR"], axis=1, inplace=True)
data.dropna(inplace=True)
data.drop_duplicates(subset="datetime", keep="last", inplace=True)


In [82]:
# train/test split
data = data[data["datetime"].dt.year < 2022]
data_train = data[data["datetime"].dt.year < 2021]
data_test = data[(data["datetime"].dt.year == 2021) 
                 & (data["datetime"].dt.month < 7)]
X_train = data_train.drop(["datetime", "priceBE"], axis=1)
y_train = data_train["priceBE"]
X_test = data_test.drop(["datetime", "priceBE"], axis=1)
y_test = data_test["priceBE"]

In [83]:
def CRPS(predictions, observations):
    results = pd.DataFrame([])
    for column in predictions:
        delta = np.array(predictions[column].values - observations.values)
        tau = (predictions.columns.get_loc(column)+1)/100
        results[predictions[column].name] = np.square(tau - np.heaviside(delta, 1))
    return results.sum(axis=1).mean()

# Gradient Boosting

In [84]:
CRPS(predictions_GB, y_test)

13.207413793103454

In [85]:
mean_pinball_loss(y_test, predictions_GB["0.05"], alpha=0.05)

1.5389535296936268

In [86]:
mean_pinball_loss(y_test, predictions_GB["0.50"], alpha=0.50)

2.61019064179013

In [88]:
mean_pinball_loss(y_test, predictions_GB["0.95"], alpha=0.95)

0.5945171318070203

# Neural networks

In [59]:
CRPS(predictions_NN, y_test)

23.924770114942532

In [89]:
mean_pinball_loss(y_test, predictions_NN["0.05"], alpha=0.05)

204.7215766862069

In [90]:
mean_pinball_loss(y_test, predictions_NN["0.50"], alpha=0.50)

477.60789712643685

In [62]:
mean_pinball_loss(y_test, predictions_NN["0.95"], alpha=0.95)

163.5103071746265

# Quantile regression

## Model 1

In [91]:
CRPS(predictions_model1, y_test)

16.21235632183909

In [92]:
mean_pinball_loss(y_test, predictions_model1["tau= 0.05"], alpha=0.05)

1.2641754079009961

In [93]:
mean_pinball_loss(y_test, predictions_model1["tau= 0.50"], alpha=0.50)

5.263316949680461

In [94]:
mean_pinball_loss(y_test, predictions_model1["tau= 0.95"], alpha=0.95)

2.2140312498370807

## Model 2

In [95]:
CRPS(predictions_model2, y_test)

18.300632183908043

In [96]:
mean_pinball_loss(y_test, predictions_model2["tau= 0.05"], alpha=0.05)

1.4765010862684647

In [97]:
mean_pinball_loss(y_test, predictions_model2["tau= 0.50"], alpha=0.50)

6.576343937116396

In [98]:
mean_pinball_loss(y_test, predictions_model3["tau= 0.95"], alpha=0.95)

1.6783950712297495

## Model 3

In [99]:
CRPS(predictions_model3, y_test)

19.373850574712648

In [100]:
mean_pinball_loss(y_test, predictions_model1["tau= 0.05"], alpha=0.05)

1.2641754079009961

In [101]:
mean_pinball_loss(y_test, predictions_model3["tau= 0.50"], alpha=0.50)

3.8663899347486703

In [102]:
mean_pinball_loss(y_test, predictions_model3["tau= 0.95"], alpha=0.95)

1.6783950712297495